This notebook will show two tricks for working with images: how to map between pixel number and RA/Dec, and how to get a smaller image "cut out" of a larger one.  We'll also show how to make a more complicated 

In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as c
from astropy.io import fits
from astropy.wcs import WCS
from reproject import reproject_interp
import ImagingHelpers as ih
from astropy.coordinates import SkyCoord
from astropy.nddata import Cutout2D

from astropy.modeling.functional_models import Const2D, Gaussian2D
from astropy.modeling import models, fitting

In [2]:
def Display(image, wcs, **kwargs):
    """ A quick way of getting an image plotted with proper WCS.
    The extra keywords get passed to imshow to adjust its behavior.  """
    fig = plt.figure()
    ax = plt.axes(projection=wcs)
    plt.subplot(ax)
    plt.imshow(image, **kwargs)
    plt.colorbar(shrink=0.8)
    plt.show()

In [3]:
hdu = fits.open('simulated_image.fits')
wcs = WCS(hdu[0].header)
data = hdu[0].data

In [4]:
Display(data, wcs, vmax=50)
plt.title('Simulated image')

Text(0.5, 1.0, 'Simulated image')

In [5]:
# The RA/Dec coordinates of a source, read off from the image above
radec0 = SkyCoord("05h30m00.9s +30d01m00s")

In [6]:
# Find the pixel coordinates from the RA/Dec in the image and display
x0, y0 = wcs.all_world2pix(radec0.ra.deg, radec0.dec.deg, 0)

In [7]:
Display(data, wcs, vmax=50)
plt.plot(x0, y0, 'x', color='red')
plt.title('Target source marked')

Text(0.5, 1.0, 'Target source marked')

In [8]:
# Get the source positions and brightnesses
x_true = hdu[1].data['x_src']
y_true = hdu[1].data['y_src']
I_true = hdu[1].data['I_src']
# Notice that we can also producet the ra, dec of the sources from the pixel positions (which is what was stored)
ra_true, dec_true = wcs.all_pix2world(x_true, y_true, 1)

In [9]:
# x and y here refer to what you would think ...
plt.imshow(data, vmax=50, origin='lower')
plt.plot(x_true, y_true, 'x', color='red')
plt.title('All sources')
plt.show()

In [10]:
# Even if displayed in RA/Dec, source positions still need to be in pixels
Display(data, wcs, vmax=50)
plt.plot(x_true, y_true, 'x', color='red')
plt.plot(x0, y0, 'x', color='green')
plt.title('All sources plus target')

Text(0.5, 1.0, 'All sources plus target')

In [11]:
# Which source in the list was this?  We need this to know the right answer for position and flux ...
# Let's find the distance between our source and all the ones in the list, and pick the source that's closest
d = np.sqrt( np.power(x_true - x0, 2) + np.power(y_true - y0, 2) )
indx_src = np.argmin(d)

In [12]:
# Check that the pixel positions and RA/Dec we read off from the image match the 
print('%.2f %.2f' % (x0, y0))
print('%.2f %.2f' % (x_true[indx_src], y_true[indx_src]))
print('%.5f %.5f' % (radec0.ra.deg, radec0.dec.deg))
print('%.5f %.5f' % (ra_true[indx_src], dec_true[indx_src]))

225.62 369.00
224.96 369.20
82.50375 30.01667
82.50402 30.01656


In [13]:
# This extent will also be good for the real data as well
extent = 0.5*u.arcmin
# The cutout object is so useful! It already has .data and .wcs attributes!
cutout = Cutout2D(data, radec0, (extent, extent), wcs=wcs, copy=True, mode='partial')

In [14]:
# Find the nominal location of the source in the cutout
cutout_max = cutout.data.max()
tpl = np.where(cutout.data == cutout_max)
y_mcu, x_mcu = tpl[0][0], tpl[1][0]

In [15]:
Display(cutout.data, cutout.wcs)
plt.plot(x_mcu, y_mcu, 'x', color='red')
plt.title('Cutout')

Text(0.5, 1.0, 'Cutout')

In [16]:
# These are pixel coordinates for the cutout, just used for fitting
x_cutout, y_cutout = np.meshgrid(np.arange(cutout.xmax_cutout+1), np.arange(cutout.ymax_cutout+1))
# OK, let's remember that Gaussian fitting ... 
gaussian = Gaussian2D(amplitude=cutout_max,x_mean=x_mcu,y_mean=y_mcu,x_stddev=3.,y_stddev=3.,theta=0.) 
offset = Const2D(amplitude=np.median(cutout.data))          

# This is the new wrinkle!
gaussoff = gaussian + offset
                       
source_model = gaussoff
source_model

<CompoundModel(amplitude_0=79.32238576, x_mean_0=29., y_mean_0=29., x_stddev_0=3., y_stddev_0=3., theta_0=0., amplitude_1=5.)>

In [17]:
# initialize a fitter
source_fitter = fitting.LevMarLSQFitter()

In [18]:
# Notice the way that the weights are passed, as 1/standard deviation
fitted_source = source_fitter(source_model, x_cutout, y_cutout, cutout.data)#, weights=np.ones_like(data)/sigma_noise)
print(fitted_source)
# We want the `cov_x` as the model covariance
#print(source_fitter.fit_info['cov_x'])
# This value will be 1 if the fit was determined to be good
print(source_fitter.fit_info['ierr'])

Model: CompoundModel
Inputs: ('x', 'y')
Outputs: ('z',)
Model set size: 1
Expression: [0] + [1]
Components: 
    [0]: <Gaussian2D(amplitude=75.18977026, x_mean=28.9271409, y_mean=29.17186002, x_stddev=3.04870624, y_stddev=2.95232446, theta=0.33240247)>

    [1]: <Const2D(amplitude=5.02307156)>
Parameters:
       amplitude_0         x_mean_0      ...      theta_0          amplitude_1   
    ----------------- ------------------ ... ------------------ -----------------
    75.18977026051256 28.927140902562666 ... 0.3324024730875609 5.023071562879702
1


In [19]:
Display(fitted_source(x_cutout, y_cutout), cutout.wcs)
plt.title('Fit')

Text(0.5, 1.0, 'Fit')

In [20]:
resid = cutout.data - fitted_source(x_cutout, y_cutout)

In [21]:
Display(resid, cutout.wcs)
plt.title('Residual')

Text(0.5, 1.0, 'Residual')

In [22]:
# Now we can go back and estimate the noise, and get an error bar for the fit!
sigma_noise = np.std(resid)
print(sigma_noise)

2.211714013603037


In [23]:
gaussian = Gaussian2D(amplitude=cutout_max,x_mean=x_mcu,y_mean=y_mcu,x_stddev=3.,y_stddev=3.,theta=0.) 
offset = Const2D(amplitude=np.median(cutout.data))                      
gaussoff = gaussian + offset
source_model = gaussoff
source_model

<CompoundModel(amplitude_0=79.32238576, x_mean_0=29., y_mean_0=29., x_stddev_0=3., y_stddev_0=3., theta_0=0., amplitude_1=5.)>

In [24]:
source_fitter = fitting.LevMarLSQFitter()
fitted_source = source_fitter(source_model, x_cutout, y_cutout, cutout.data, weights=np.ones_like(cutout.data)/sigma_noise)
print(fitted_source)
# We want the `cov_x` as the model covariance
cov = source_fitter.fit_info['cov_x']
print(cov)
# This value will be 1 if the fit was determined to be good
print(source_fitter.fit_info['ierr'])

Model: CompoundModel
Inputs: ('x', 'y')
Outputs: ('z',)
Model set size: 1
Expression: [0] + [1]
Components: 
    [0]: <Gaussian2D(amplitude=75.18977026, x_mean=28.9271409, y_mean=29.17186002, x_stddev=3.04870624, y_stddev=2.95232446, theta=0.33240247)>

    [1]: <Const2D(amplitude=5.02307156)>
Parameters:
       amplitude_0         x_mean_0      ...      theta_0          amplitude_1   
    ----------------- ------------------ ... ------------------ -----------------
    75.18977026051326 28.927140902562716 ... 0.3324024731348911 5.023071562879782
[[ 3.45946439e-01  8.29350294e-06  8.35272853e-06 -7.01347861e-03
  -6.79214198e-03  4.65077688e-08 -2.50731462e-12]
 [ 8.29350294e-06  5.65023708e-04  1.09137620e-05 -2.92950025e-07
  -4.19720309e-08  1.04653272e-06  6.32451849e-12]
 [ 8.35272853e-06  1.09137620e-05  5.37237100e-04 -3.91621501e-08
  -2.90076522e-07 -1.00280274e-06  6.42309406e-12]
 [-7.01347861e-03 -2.92950025e-07 -3.91621501e-08  5.78334069e-04
   9.23565307e-06  1.17852595e

In [25]:
# Finally, let's compare the fitted amplitude to the true one
print('True amplitude %.1f' % I_true[indx_src])
print('Fitted amplitude %.1f +/- %.1f' %(fitted_source.amplitude_0.value, np.sqrt(cov[0,0])))

True amplitude 75.5
Fitted amplitude 75.2 +/- 0.6
